<a href="https://colab.research.google.com/github/ankitarm/SQL_Data_Engineer/blob/main/SQL_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Find service consistently down for more than 3 minutes.

| service\_name | updated\_time | status |
| ------------- | ------------- | ------ |
| hdfs          | 10:02         | down   |
| hdfs          | 10:03         | down   |
| hdfs          | 10:04         | down   |
| hdfs          | 10:05         | down   |
| hdfs          | 10:06         | up     |

You need to:

Detect if a service is "down" for more than 3 minutes.

Then output:

service_name

start_time (first down)

end_time (when it comes up)

duration (only if > 3 mins)
```
WITH down_only AS (
  SELECT *
  FROM tracker
  WHERE status = 'down'
),
numbered AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY service_name ORDER BY updated_time) AS rn
  FROM down_only
),
grouped AS (
  SELECT *,
         updated_time - INTERVAL '1 minute' * rn AS grp_key
  FROM numbered
),
aggregated AS (
  SELECT service_name,
         MIN(updated_time) AS start_time,
         MAX(updated_time) AS end_time,
         COUNT(*) AS down_count,
         'down' AS status
  FROM grouped
  GROUP BY service_name, grp_key
)
SELECT service_name, start_time, end_time, status
FROM aggregated
WHERE EXTRACT(EPOCH FROM end_time - start_time) / 60 > 3;
```
| Part                                      | Meaning                                                                                                                           |
| ----------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------- |
| `updated_time`                            | A timestamp showing when the service was checked.                                                                                 |
| `rn`                                      | A row number assigned to each row **ordered by updated\_time**.                                                                   |
| `INTERVAL '1 minute' * rn`                | Converts the row number into a **time offset** (1 min, 2 min, etc.).                                                              |
| `updated_time - INTERVAL '1 minute' * rn` | Offsets the timestamp backward by the row number → this **normalizes consecutive times** to the same base, so you can group them. |

10:02 - 1 min → 10:01

10:03 - 2 min → 10:01

10:04 - 3 min → 10:01